# SPARQL Workshop - Section 06: Query Explain

Troubleshooting query performance is something every graph database developer will need to do at some point.  Thankfully, Neptune provides a couple of different mechanisms for seeing how a query is executed.  In the following section we will cover different methods for exposing query execution timing and performance.

Happy Traversing!

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>


### 06.01 - Using SPARQL EXPLAIN

Inside of Neptune is a query parser and optimizer that we at AWS have custom built for both the Apache TinkerPop and RDF graph frameworks and their respective query languages (Gremlin and SPARQL respectively).  In order to expose a little more about what the optimizer is doing, we have exposed a parameter for the SPARQL REST endpoint that would output the optimized query plan (Explain) for the given query.  Below is an example of how to use the SPARQL Query Explainer in Neptune.

Additional Documentation:
- Using the SPARQL Explain API in Neptune - https://docs.aws.amazon.com/neptune/latest/userguide/sparql-explain.html
- SPARQL Explain Blog Post - https://aws.amazon.com/blogs/database/using-sparql-explain-to-understand-query-execution-in-amazon-neptune/

Here we can pass our original queries to the SPARQL API and ask for the explain.  First, let's start off with a given query.  In this next case we are attempting to find all genres of productions that Harrison Ford has performed in:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT DISTINCT ?genre WHERE 
{
    ?harrison rdf:type class:Artist .
    ?harrison dataProp:name "Harrison Ford" .
    ?movies objProp:actor ?harrison .
    ?movies objProp:genre ?genre
}

Next, let's take a look at feeding this query into the SPARQL explain API.  We've saved our query as a text string below and have fed it into a curl command with the SPARQL explain API endpoint.

In [ ]:
%%sparql explain

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT DISTINCT ?genre WHERE
{
    ?harrison rdf:type class:Artist .
    ?harrison dataProp:name "Harrison Ford" .
    ?movies objProp:actor ?harrison .
    ?movies objProp:genre ?genre
}

The output above is a little awkward to read.  You can attempt to use the tabular output by changing the Accept header on the curl request to `text/plain`, however the table is usually wider than what Jupyter can support.

The output shows the breakdown of the query as it is processed within Neptune.  The key thing to look at here is the number of `Units In` and `Units Out`.  This corresponds to the number of triples that are being referenced during the query.  The trick here is to minimize the number of potential triples being assessed in order to optimize your queries.

Let's look at a different version of this same query, but queried from a different point of view.  I'm also injecting a Query Hint into this query to force Neptune to not reorder the evaluation order of the triples.  The optimizer has the ability to reorder the evaluation of triples for better performance.  Here I want to display what poor performance looks like via the SPARQL query explainer.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX hint: <http://aws.amazon.com/neptune/vocab/v01/QueryHints#>
        
SELECT DISTINCT ?genre 
WHERE 
{
    hint:Query hint:joinOrder "Ordered" .
    ?genre rdf:type class:Genre .
    ?movies objProp:genre ?genre .
    ?movies objProp:actor ?harrison .
    ?harrison dataProp:name "Harrison Ford" .
}

In [ ]:
%%sparql explain

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX hint: <http://aws.amazon.com/neptune/vocab/v01/QueryHints#>
        
SELECT DISTINCT ?genre 
WHERE 
{
    hint:Query hint:joinOrder "Ordered" .
    ?genre rdf:type class:Genre .
    ?movies objProp:genre ?genre .
    ?movies objProp:actor ?harrison .
    ?harrison dataProp:name "Harrison Ford" .
}

Notice in this output that the number of triples being evaluated via the `Units In` and `Units Out` is MUCH larger than in the previous version of the query.  This just goes to show that order of evaluation matters.  You can change the direction of your query to support lower cardinality entities in order to make your queries more performant.

### Review

In the preceding noteboook we covered how to use Neptune's SPARQL xplain APIs to look at the performance of queries.  These can be used during the process to optimize queries for better performance.

### End of Section 06 - Move onto the next notebook.